In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import datetime

In [146]:
mu_masse_1 = 5.944892676091842
s_masse_1 = 1.0452954216771342
mu_masse_2 = 4.012421689011621
s_masse_2 = 1.2811831375376432
mu_speed_1 = 8.788235294117646
s_speed_1 = 1.9745088303442118
mu_speed_2 = 37.790625
s_speed_2 = 5.31080027956004
alpha_t_delta_1 = 0.7720943018929209
beta_t_delta_1 = 32.180388904581854
alpha_t_delta_2 = 0.6924339239082432
beta_t_delta_2 = 85.33142881264439

In [147]:
def random_masse_1():
    masse_1 = np.random.lognormal(mu_masse_1, s_masse_1, 1).round(1)
    return masse_1

def random_masse_2():
    masse_2 = np.random.lognormal(mu_masse_2, s_masse_2, 1).round(1)
    return masse_2

def random_speed_1():
    speed_1 = np.random.normal(mu_speed_1, s_speed_1, 1).round(2)
    return speed_1

def random_speed_2():
    speed_2 = np.random.normal(mu_speed_2, s_speed_2, 1).round(2)
    return speed_2

def random_t_delta_1():
    t_delta_1 = int(np.random.gamma(alpha_t_delta_1, beta_t_delta_1, 1).round(0))
    return t_delta_1

def random_t_delta_2():
    t_delta_2 = int(np.random.gamma(alpha_t_delta_2, beta_t_delta_2, 1).round(0))
    return t_delta_2
    

In [148]:
results = []
double_entries = []

amount_of_hours = 200

for i in range (0, amount_of_hours, 1):
    results.append(i)

for i in range (0, amount_of_hours, 1):
    double_entries.append(i)
    
    
results = pd.DataFrame(results, columns= ['Hours'])
double_entries = pd.DataFrame(double_entries, columns= ['Hours'])

header = ['Day', 'Masse 1', 'Speed 1', 'Masse 2', 'Speed 2', 'Energie in kJ 1', 'Energie in kJ 2']

for value in header:
    results[value] = np.nan
    
for value in header: 
    double_entries[value] = np.nan
    

In [149]:
for value in header:
    results[value] = np.nan

In [150]:
end = amount_of_hours
start = 0

i = 0
j = 0

event_1 = 0
event_2 = 0

In [151]:
while i < end:
    t_delta_1 = random_t_delta_1()
    if t_delta_1 > 0:        
        event_1 += t_delta_1
        
        masse_1 = random_masse_1()
        results['Masse 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        results['Speed 1'][event_1] = speed_1
        print("i %i" % event_1)    
        i += t_delta_1
    else: 
        masse_1 = random_masse_1()
        double_entries['Masse 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        double_entries['Speed 1'][event_1] = speed_1
        print("double i %i" % event_1)

        
while j < end:
    t_delta_2 = random_t_delta_2()
    if t_delta_2 > 0:
        event_2 += t_delta_2
    
        masse_2 = random_masse_2()
        results['Masse 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        results['Speed 2'][event_2] = speed_2
        print("j %i" % event_2)
    
        j += t_delta_2
    
    else:
        masse_2 = random_masse_2()
        double_entries['Masse 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        double_entries['Speed 2'][event_2] = speed_2 
        print("double j %i" % event_2)

i 22
i 39
i 67
i 87
i 164
i 168
i 172
i 190
i 210
j 45
double j 45
j 157
j 418


c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [152]:
filtered_double_entries = double_entries[(double_entries['Energie in kJ 1'] != 0) & (double_entries['Energie in kJ 2'] != 0)]


In [153]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(filtered_double_entries)


     Hours  Day  Masse 1  Speed 1  Masse 2  Speed 2  Energie in kJ 1  \
0        0  NaN      NaN      NaN      NaN      NaN              NaN   
1        1  NaN      NaN      NaN      NaN      NaN              NaN   
2        2  NaN      NaN      NaN      NaN      NaN              NaN   
3        3  NaN      NaN      NaN      NaN      NaN              NaN   
4        4  NaN      NaN      NaN      NaN      NaN              NaN   
5        5  NaN      NaN      NaN      NaN      NaN              NaN   
6        6  NaN      NaN      NaN      NaN      NaN              NaN   
7        7  NaN      NaN      NaN      NaN      NaN              NaN   
8        8  NaN      NaN      NaN      NaN      NaN              NaN   
9        9  NaN      NaN      NaN      NaN      NaN              NaN   
10      10  NaN      NaN      NaN      NaN      NaN              NaN   
11      11  NaN      NaN      NaN      NaN      NaN              NaN   
12      12  NaN      NaN      NaN      NaN      NaN             

In [154]:
# Berechnung fehlender Felder


results['Energie in kJ 1'] = results.apply(
    lambda x: results['Masse 1'] * results['Speed 1'] * results['Speed 1'] / 2 / 1000
)

results['Energie in kJ 2'] = results.apply(
    lambda x: results['Masse 2'] * results['Speed 2'] * results['Speed 2'] / 2 / 1000
)

results['Day'] = results.apply(
    lambda x: results['Hours'] // 24
)